# Chapter 11 - Classification and Regression Trees

### Gini Index

In [2]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # Count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each groups
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [3]:
# test Gini values
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


### Create Split

#### Splitting a Dataset

In [4]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

#### Evaluating All Splits

In [7]:
# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            print( 'X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return { 'index' :b_index, 'value' :b_value, 'groups' :b_groups}

In [8]:
# Test getting the best split
dataset = [[2.771244718,1.784783929,0],
[1.728571309,1.169761413,0],
[3.678319846,2.81281357,0],
[3.961043357,2.61995032,0],
[2.999208922,2.209014212,0],
[7.497545867,3.162953546,1],
[9.00220326,3.339047188,1],
[7.444542326,0.476683375,1],
[10.12493903,3.234550982,1],
[6.642287351,3.319983761,1]]
split = get_split(dataset)
print( 'Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

X1 < 2.771 Gini=0.444
X1 < 1.729 Gini=0.500
X1 < 3.678 Gini=0.286
X1 < 3.961 Gini=0.167
X1 < 2.999 Gini=0.375
X1 < 7.498 Gini=0.286
X1 < 9.002 Gini=0.375
X1 < 7.445 Gini=0.167
X1 < 10.125 Gini=0.444
X1 < 6.642 Gini=0.000
X2 < 1.785 Gini=0.500
X2 < 1.170 Gini=0.444
X2 < 2.813 Gini=0.320
X2 < 2.620 Gini=0.417
X2 < 2.209 Gini=0.476
X2 < 3.163 Gini=0.167
X2 < 3.339 Gini=0.444
X2 < 0.477 Gini=0.500
X2 < 3.235 Gini=0.286
X2 < 3.320 Gini=0.375
Split: [X1 < 6.642]


### Build a Tree

#### Terminal Nodes

In [9]:
# Create a terminal node values
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

#### Recursive Splitting

In [11]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(left) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

#### Building a Tree

In [12]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [13]:
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth* ' ' , (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth* ' ' , node)))

In [16]:
dataset = [[2.771244718,1.784783929,0],
[1.728571309,1.169761413,0],
[3.678319846,2.81281357,0],
[3.961043357,2.61995032,0],
[2.999208922,2.209014212,0],
[7.497545867,3.162953546,1],
[9.00220326,3.339047188,1],
[7.444542326,0.476683375,1],
[10.12493903,3.234550982,1],
[6.642287351,3.319983761,1]]
tree = build_tree(dataset, 3, 1)
print_tree(tree)

X1 < 2.771 Gini=0.444
X1 < 1.729 Gini=0.500
X1 < 3.678 Gini=0.286
X1 < 3.961 Gini=0.167
X1 < 2.999 Gini=0.375
X1 < 7.498 Gini=0.286
X1 < 9.002 Gini=0.375
X1 < 7.445 Gini=0.167
X1 < 10.125 Gini=0.444
X1 < 6.642 Gini=0.000
X2 < 1.785 Gini=0.500
X2 < 1.170 Gini=0.444
X2 < 2.813 Gini=0.320
X2 < 2.620 Gini=0.417
X2 < 2.209 Gini=0.476
X2 < 3.163 Gini=0.167
X2 < 3.339 Gini=0.444
X2 < 0.477 Gini=0.500
X2 < 3.235 Gini=0.286
X2 < 3.320 Gini=0.375
X1 < 2.771 Gini=0.000
X1 < 1.729 Gini=0.000
X1 < 3.678 Gini=0.000
X1 < 3.961 Gini=0.000
X1 < 2.999 Gini=0.000
X2 < 1.785 Gini=0.000
X2 < 1.170 Gini=0.000
X2 < 2.813 Gini=0.000
X2 < 2.620 Gini=0.000
X2 < 2.209 Gini=0.000
X1 < 7.498 Gini=0.000
X1 < 9.002 Gini=0.000
X1 < 7.445 Gini=0.000
X1 < 10.125 Gini=0.000
X1 < 6.642 Gini=0.000
X2 < 3.163 Gini=0.000
X2 < 3.339 Gini=0.000
X2 < 0.477 Gini=0.000
X2 < 3.235 Gini=0.000
X2 < 3.320 Gini=0.000
X1 < 7.445 Gini=0.000
X1 < 6.642 Gini=0.000
X2 < 0.477 Gini=0.000
X2 < 3.320 Gini=0.000
X1 < 7.498 Gini=0.000
X1 < 9.0

### Make a prediction

In [17]:
# Make a prediction with a decision tree
def predict(node, row):
     if row[node['index']] < node['value']:
         if isinstance(node['left'], dict):
             return predict(node['left'], row)
         else:
             return node['left']
     else:
         if isinstance(node['right'], dict):
             return predict(node['right'], row)
         else:
             return node['right']

In [18]:
# contrived dataset
dataset = [[2.771244718,1.784783929,0],
[1.728571309,1.169761413,0],
[3.678319846,2.81281357,0],
[3.961043357,2.61995032,0],
[2.999208922,2.209014212,0],
[7.497545867,3.162953546,1],
[9.00220326,3.339047188,1],
[7.444542326,0.476683375,1],
[10.12493903,3.234550982,1],
[6.642287351,3.319983761,1]]
# predict with a stump
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left' : 0}
for row in dataset:
    prediction = predict(stump, row)
    print('Expected=%d, Got=%d'%(row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


### Banknote Case Study

In [19]:
from random import seed

from Codes.ch01_load_and_convert_data import load_csv, str_column_to_float
from Codes.ch06_algorithm_test_harnesses import evaluate_algorithm_kfold

In [23]:
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [24]:
# Test CART on Bank Note dataset
seed(1)

# load and prepare data
filename = './data/data_banknote_authentication.csv'
dataset = load_csv(filename)

# convert string attributes to integers
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)

# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm_kfold(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

2 < 2.576 Gini=0.000
X2 < 0.213 Gini=0.000
X2 < 3.361 Gini=0.000
X2 < 10.357 Gini=0.000
X2 < -3.936 Gini=0.000
X2 < -3.584 Gini=0.000
X2 < 3.329 Gini=0.000
X2 < 8.670 Gini=0.000
X2 < 1.148 Gini=0.000
X2 < 9.995 Gini=0.000
X2 < 7.131 Gini=0.000
X2 < 4.302 Gini=0.000
X2 < 6.870 Gini=0.000
X2 < 5.219 Gini=0.000
X2 < -0.739 Gini=0.000
X2 < 0.662 Gini=0.000
X2 < 6.709 Gini=0.000
X2 < 9.010 Gini=0.000
X2 < -6.317 Gini=0.000
X2 < 8.995 Gini=0.000
X2 < 8.617 Gini=0.000
X2 < -3.073 Gini=0.000
X2 < 11.112 Gini=0.000
X2 < -0.993 Gini=0.000
X2 < 7.402 Gini=0.000
X2 < 2.107 Gini=0.000
X2 < 5.391 Gini=0.000
X2 < 6.986 Gini=0.000
X2 < 3.512 Gini=0.000
X2 < 1.548 Gini=0.000
X2 < 6.878 Gini=0.000
X2 < 1.707 Gini=0.000
X2 < -4.815 Gini=0.000
X2 < -2.040 Gini=0.000
X2 < 8.960 Gini=0.000
X2 < 6.858 Gini=0.000
X2 < 3.013 Gini=0.000
X2 < 8.722 Gini=0.000
X2 < 8.633 Gini=0.000
X2 < 10.053 Gini=0.000
X2 < 6.822 Gini=0.000
X2 < 9.887 Gini=0.000
X2 < -4.060 Gini=0.000
X2 < 10.146 Gini=0.000
X2 < 2.622 Gini=0.00

## Future Works

* Algorithm Tuning. The application of CART to the Bank Note dataset was not tuned. Experiment with different parameter values and see if you can achieve better performance.
* Cross Entropy. Another cost function for evaluating splits is cross entropy (logloss). You could implement and experiment with this alternative cost function.
* Tree Pruning. An important technique for reducing overfitting of the training dataset is to prune the trees. Investigate and implement tree pruning methods.
* Categorical Dataset. The example was designed for input data with numerical or ordinal input attributes, experiment with categorical input data and splits that may use equality instead of ranking.
* Regression. Adapt the tree for regression using a different cost function and method for creating terminal nodes.
* More Datasets. Apply the algorithm to more datasets on the UCI Machine Learning Repository.